<a href="https://colab.research.google.com/github/fdac25/football/blob/main/football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kagglehub

In [2]:
# Import the dataset. Too big to include in GitHub so have to download it to colab

# Download latest version
path = kagglehub.dataset_download("philiphyde1/nfl-stats-1999-2022") # To access just use path variable

print("Path to dataset files:", path)

!ls -lh {path}

Using Colab cache for faster access to the 'nfl-stats-1999-2022' dataset.
Path to dataset files: /kaggle/input/nfl-stats-1999-2022
total 209M
-rw-r--r-- 1 1000 1000  63M Oct 14 14:47 weekly_player_stats_defense.csv
-rw-r--r-- 1 1000 1000 101M Oct 14 14:47 weekly_player_stats_offense.csv
-rw-r--r-- 1 1000 1000 1.7M Oct 14 14:47 weekly_team_stats_defense.csv
-rw-r--r-- 1 1000 1000 3.6M Oct 14 14:47 weekly_team_stats_offense.csv
-rw-r--r-- 1 1000 1000  13M Oct 14 14:47 yearly_player_stats_defense.csv
-rw-r--r-- 1 1000 1000  27M Oct 14 14:47 yearly_player_stats_offense.csv
-rw-r--r-- 1 1000 1000 187K Oct 14 14:47 yearly_team_stats_defense.csv
-rw-r--r-- 1 1000 1000 441K Oct 14 14:47 yearly_team_stats_offense.csv


In [8]:
#Display the first few rows of the dataset to understand its structure and contents.
df_yearly_player_stats = pd.read_csv(path + "/yearly_player_stats_offense.csv")
df_yearly_player_stats.head()

,player_id,player_name,position,birth_year,draft_year,draft_round,draft_pick,draft_ovr,height,weight,...,delta_comp_pct,delta_int_pct,delta_pass_td_pct,delta_ypa,delta_yptarget,delta_ypr,delta_rush_td_pct,delta_ypc,delta_td_pct,delta_yptouch
0,00-0000865,Charlie Batch,QB,1974,1998.0,2.0,30.0,60.0,74.0,216.0,...,0.220036,-1.0,-1.0,0.025641,0.000000,0.000000,0.0,0.000000,-0.909091,-0.388795
1,00-0004541,Donald Driver,WR,1975,1999.0,7.0,7.0,213.0,72.0,194.0,...,0.000000,NaN,NaN,0.000000,-0.172028,-0.068732,0.0,0.000000,49.000000,1.226852
2,00-0006101,Tony Gonzalez,TE,1976,1997.0,1.0,13.0,13.0,77.0,247.0,...,0.000000,0.0,0.0,0.000000,0.273859,-0.216170,0.0,0.000000,3.833333,0.236242
3,00-0006101,Tony Gonzalez,TE,1976,1997.0,1.0,13.0,13.0,77.0,247.0,...,0.000000,NaN,NaN,0.000000,-0.165038,0.085776,0.0,0.000000,-0.413793,0.085776
4,00-0007091,Matt Hasselbeck,QB,1975,1998.0,6.0,34.0,187.0,76.0,235.0,...,0.149281,-1.0,-1.0,-0.162698,0.000000,0.000000,-1.0,-0.337868,0.500000,0.250509


In [6]:
df_yearly_team_stats = pd.read_csv(path + "/yearly_team_stats_offense.csv")
df_yearly_team_stats.head()

,team,season,season_type,shotgun,no_huddle,qb_dropback,qb_scramble,total_off_yards,pass_attempts,complete_pass,...,yptarget,ayptarget,ypr,rush_td_pct,ypc,touches,total_tds,td_pct,total_yards,yptouch
0,ARI,2012,REG,605.0,56.0,676.0,12.0,4587,587.0,337.0,...,5.76,8.28,10.04,0.028,3.42,1276.0,32.0,0.03,7970.0,6.25
1,ATL,2012,POST,64.0,3.0,80.0,2.0,894,74.0,54.0,...,8.73,12.43,11.96,0.000,5.06,177.0,12.0,0.07,1540.0,8.70
2,ATL,2012,REG,418.0,101.0,664.0,20.0,6116,601.0,422.0,...,7.85,8.33,11.18,0.032,3.69,1402.0,76.0,0.05,10835.0,7.73
3,BAL,2012,POST,97.0,53.0,134.0,2.0,1679,126.0,73.0,...,9.05,12.23,15.62,0.036,3.88,338.0,27.0,0.08,2819.0,8.34
4,BAL,2012,REG,452.0,257.0,608.0,9.0,5897,549.0,334.0,...,7.28,10.01,11.96,0.043,4.28,1327.0,63.0,0.05,9893.0,7.46


In [11]:
# Filter the dataframe to keep only the columns we need
df_yearly_player_stats_filtered = df_yearly_player_stats[['season_complete_pass', 'season_passing_yards', 'season_receiving_yards', 'season_rush_attempts', 'season_rushing_yards', 'season_fantasy_points_ppr']]
df_yearly_player_stats_filtered.head()

,season_complete_pass,season_passing_yards,season_receiving_yards,season_rush_attempts,season_rushing_yards,season_fantasy_points_ppr
0,45.0,475.0,0.0,0.0,0.0,17.00
1,0.0,0.0,77.0,0.0,0.0,27.70
2,0.0,0.0,129.0,0.0,0.0,36.90
3,0.0,0.0,930.0,0.0,0.0,228.00
4,138.0,1367.0,0.0,13.0,38.0,87.48
